In [0]:
!pip install h5py
!pip install tensorflow-gpu
!pip install wandb

In [0]:
import h5py
import wandb
import numpy as np
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from matplotlib import pyplot
from wandb.keras import WandbCallback
tf.test.gpu_device_name()

## Loading data

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
training_data = h5py.File('/content/drive/My Drive/Colab Notebooks/Lego classification/Data/legobricks_data_withoutBot.h5', 'r')
rwColorData = h5py.File('/content/drive/My Drive/Colab Notebooks/Lego classification/Data/legobricks_rwColorAug.h5', 'r')
rwGreyscaleData = h5py.File('/content/drive/My Drive/Colab Notebooks/Lego classification/Data/legobricks_rwGrayAug.h5', 'r')

In [0]:
print(list(training_data.keys()))
print(list(rwColorData.keys()))
print(list(rwGreyscaleData.keys()))

In [0]:
#Training data
x_train = training_data['x_train'][()]
y_train = training_data['y_train'][()]
x_val = training_data['x_val'][()]
y_val = training_data['y_val'][()]
x_test = training_data['x_test'][()]
y_test = training_data['y_test'][()]

#Real world Lego images
x_rwColor = rwColorData['x_test'][()]
y_rwColor = rwColorData['y_test'][()]
x_rwGreyscale = rwGreyscaleData['x_test'][()]
y_rwGreyscale = rwGreyscaleData['y_test'][()]

In [0]:
#Trying out fitting with images of edges

def canny_edge(img):
  canny = cv2.Canny(np.uint8(img), 50, 100)
  edges = np.dstack((canny, canny, canny))
  edges = np.true_divide(edges, 255)
  return edges

for i in range(len(x_train)):
    x_train[i] = canny_edge(x_train[i])

for i in range(len(x_rwGreyscale)):
    x_rwGreyscale[i] = canny_edge(x_rwGreyscale[i])

In [0]:
#Data augmentation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=[-30, 30],
    height_shift_range=[-30, 30],
    shear_range=0.5)

train_data = train_datagen.flow(x_train, y_train, batch_size=64)
#val_data = train_datagen.flow(x_val, y_val, batch_size=50, shuffle=True)
#test_data = train_datagen.flow(x_test, y_test, batch_size=50, shuffle=True)

#rwColorTest_data = train_datagen.flow(x_rwColor, y_rwColor, batch_size=64)
#rwGreyTest_data = val_datagen.flow(x_rwGreyscale, y_rwGreyscale, batch_size=50, shuffle=True)

In [0]:
#Testing data generator

def canny_edge(img):
  canny = cv2.Canny(np.uint8(img), 50, 100)
  edges = np.dstack((canny, canny, canny))
  edges = np.true_divide(edges, 255)
  print(np.amax(edges))
  return edges

#Image data generator
datagenerator = ImageDataGenerator(
    zoom_range=[1.5, 1.5],
    preprocessing_function=canny_edge)

#datagenerator = ImageDataGenerator(
#    rescale=1./255,
#    width_shift_range=[-50, 50],
#    height_shift_range=[-50, 50],
#    horizontal_flip=True,
#    vertical_flip=True,
#    rotation_range=360,
#    brightness_range=[0.9, 1.1],
#		zoom_range=[1.5, 1.5])

realWorld_ColorData = datagenerator.flow(x_rwGreyscale, batch_size=1, shuffle=True)

# generate samples and plot
for i in range(9):
    # define subplot
    plt.subplot(330 + 1 + i)
    img = realWorld_ColorData.next()
    img = np.reshape(img, (224, 224, 3))
    plt.imshow(img, cmap=pyplot.get_cmap('gray'))

# show the figure
plt.show()

## Models

In [0]:
def create_MobileNet(num_of_neurons):
    URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
    mobileNet = hub.KerasLayer(URL, input_shape=(224, 224, 3), trainable=False)

    model = tf.keras.Sequential([
        mobileNet,
        tf.keras.layers.Dense(num_of_neurons, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
        ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
        )

    return model

In [0]:
def create_ResNet_v2_152():
    URL = "https://tfhub.dev/google/imagenet/resnet_v2_152/classification/4"
    ResNet = hub.KerasLayer(URL, input_shape=(224, 224, 3), trainable=False)

    model = tf.keras.Sequential([
        ResNet,
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

In [0]:
def create_MobileNetFineTune():
    URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
    mobileNet = hub.KerasLayer(URL, input_shape=(224, 224, 3), trainable=True)

    model = tf.keras.Sequential([
        mobileNet,
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])

    return model

## Running model

In [0]:
#Create model and print it's summary

model = create_MobileNet(256)
model.summary()

In [0]:
#Initialize run on wandb

run = wandb.init(project="Lego classification", entity="damirj", name="MobileNet-256-UpdatedDataset-shear-0.5")

In [0]:
# Model training

epochs = 50
history = model.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_data=(x_val, y_val), callbacks=[WandbCallback()])

In [0]:
wandb.log({"Real world image accuracy":model.evaluate(rwColorTest_data, verbose=1)[1]})

In [0]:
#Ploting confusion matrix

prediction = model.predict_classes(x_rwColor)

!pip install mlxtend

from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix

mat = confusion_matrix(y_rwColor, prediction)
plot_confusion_matrix(conf_mat=mat)

In [0]:
wandb.log({"ResNet-1024-Aug": plot_confusion_matrix(conf_mat=mat)})

# Saving model (.h5, tflite, tfjs)

In [0]:
#Save model to .h5
model.save("/content/drive/My Drive/Colab Notebooks/Lego classification/H5/test1", save_format='tf')

#tf.keras.experimental.export_saved_model(model, '/content/drive/My Drive/Colab Notebooks/Lego classification/H5/mobilenetv2_adam_256.h5')

In [0]:
#Convert and save model to .tflite
import pathlib

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path("/content/drive/My Drive/Colab Notebooks/Lego classification/TFLite/mobilenetv2_adam_256.tflite")
tflite_model_file.write_bytes(tflite_model)

In [0]:
#Convert and save model for tensorflow.js
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "/content/drive/My Drive/Colab Notebooks/Lego classification/TFjs/mobilenetv2_adam_256")